# Classes

## Event List

In [ ]:
class Event_List:
  def __init__(self):
    self.event = {}

  def __setitem__(self, key, value):
    if key in self.event.keys():
      self.event[key].append(value)
    else:
      self.event[key]=[value]

  def __getitem__(self, key):
    e_list = [v for k,v in sorted(self.event.items(),key = lambda x:x[0])]
    e_list = [item for sublist in e_list for item in sublist]
    return e_list[key]

  def pop(self,key=0):
    if self.is_empty():
      return None

    e_list = [(k,v) for k,v in sorted(self.event.items(),key = lambda x:x[0])]
    e_list = [(sublist[0],item) for sublist in e_list for item in sublist[1]]

    target = e_list[key]
    del self[key]
    return target

  def top(self,key=0):
    if self.is_empty():
      return None

    e_list = [(k,v) for k,v in sorted(self.event.items(),key = lambda x:x[0])]
    e_list = [(sublist[0],item) for sublist in e_list for item in sublist[1]]

    target = e_list[key]
    return target

  def __delitem__(self, key):
    e_list = [(k,v) for k,v in sorted(self.event.items(),key = lambda x:x[0])]
    e_list = [sublist[0] for sublist in e_list for item in sublist[1]]
    target = e_list[key]
    tmp = [ i for i,n in enumerate(e_list) if n==target]
    oc = tmp.index(key)
    self.event[target].pop(oc)
    if len(self.event[target])==0:
      self.event.pop(target)

  def __str__(self):
    e_list = [(k,v) for k,v in sorted(self.event.items(),key = lambda x:x[0])]
    return e_list.__str__()

  def is_empty(self):
    return len(self.event) == 0


  def first_event(self,event,pm_id):
    e_list = [(k,v) for k,v in sorted(self.event.items(),key = lambda x:x[0])]
    e_list = [(sublist[0],item) for sublist in e_list for item in sublist[1]]
    for k,v in e_list:
      if v[0]==event and v[1] == pm_id:
        return k

## Function

In [ ]:
class Function:
  def __init__(self,t,id,execution_time,ram,cpu,gpu=0,tpu=0,runtime='python',priority=1,deadline=None):
    self.t = t
    self.id = id
    self.execution_time = execution_time
    if deadline == None:
      self.deadline = 100 * execution_time
    else:
      self.deadline = deadline
    self.ram = ram
    self.cpu = cpu
    self.gpu = gpu
    self.tpu = tpu
    self.runtime = runtime
    self.priority = priority
    self.finished_at = None

  def __str__(self):
    return f'Function(id={self.id},RAM={self.ram},CPU={self.cpu},GPU={self.gpu},TPU={self.tpu},Entered at {self.t},Finished at {self.finished_at},Execution time={self.execution_time},Deadline={self.deadline},Runtime={self.runtime})'

  def __repr__(self):
    return self.__str__()

## Container

In [ ]:
class Container:
  def __init__(self,id,ram,cpu,gpu,tpu,runtime='python',status='COLD',fx=None):
    self.id = id
    self.execution_time = 0
    self.ram = ram
    self.cpu = cpu
    self.gpu = gpu
    self.tpu = tpu
    self.status = status
    self.runtime = runtime
    self.fx = fx

    self.waited_clock = None

  def inject(self,fx):
    self.fx = fx

  def __str__(self):
    if self.fx == None:
      return f'Container(id={self.id},RAM={self.ram},CPU={self.cpu},GPU={self.gpu},TPU={self.tpu},Execution time={self.execution_time})'
    else:
      return f'Container(id={self.id},RAM={self.ram},CPU={self.cpu},GPU={self.gpu},TPU={self.tpu},Execution time={self.execution_time},{self.fx})'

  def __repr__(self):
    return self.__str__()

## Physical Machine

In [ ]:
class PM:
  def __init__(self,id,ram,cpu,gpu=0,tpu=0):
    self.id = id
    self.ram = ram
    self.cpu = cpu
    self.gpu = gpu
    self.tpu = tpu

    self.used_ram = 0
    self.used_cpu = 0
    self.used_gpu = 0
    self.used_tpu = 0

    self.functions_queue = []
    self.containers_queue = []

    self.completed_functions=[]
    self.missed_functions=[]
    self.rejected_functions=[]
    self.completed_containers=[]

    self.ram_utilization = 0
    self.cpu_utilization = 0
    self.gpu_utilization = 0
    self.tpu_utilization = 0

    self.ram_unit_price = None
    self.cpu_unit_price = None
    self.gpu_unit_price = None
    self.tpu_unit_price = None

    self.container_warm_time = None
    self.cold_start_delay = None
    self.warm_start_delay = None
    self.steal_warm_delay = None
    self.cpu_over_gpu = None
    self.cpu_over_tpu = None
    self.verbose = False

    self.penalty_coeff = None
    self.cost_coeff = None
    self.conversion_allowed = None
    self.steal_warm_allowed = None

    self.cold_starts = 0
    self.warm_starts = 0

  def __str__(self):
    return f'PM(id={self.id},RAM={self.ram},CPU={self.cpu},GPU={self.gpu},TPU={self.tpu})'

  def __repr__(self):
    return self.__str__()

  def add_function(self,fx: Function):
    self.functions_queue.append(fx)

  def remove_function(self,fx):
    self.functions_queue = [x for x in self.functions_queue if x != fx]



  def get_available_resources(self):
    r = self.ram - self.used_ram
    c = self.cpu - self.used_cpu
    g = self.gpu - self.used_gpu
    t = self.tpu - self.used_tpu

    return r,c,g,t

  def get_functions_to_schedule(self):
    fx_list = []

    available_ram, available_cpu, available_gpu, available_tpu, = self.get_available_resources()
    for fx in self.functions_queue:
      if fx.ram <= available_ram and fx.cpu <= available_cpu and fx.gpu <= available_gpu and fx.tpu <= available_tpu:
        fx_list.append(fx)
        available_ram -= fx.ram
        available_cpu -= fx.cpu
        available_gpu -= fx.gpu
        available_tpu -= fx.tpu
    
    return fx_list

  def get_warm_container(self,fx):
    appropriate_warm_containers = [c for c in self.containers_queue if (c.status == 'WARM' and c.ram >=fx.ram and c.cpu >= fx.cpu and c.gpu >= fx.gpu and c.tpu >= fx.tpu and c.runtime == fx.runtime)]
    if len(appropriate_warm_containers)>0:
      for app in appropriate_warm_containers:
        if app.fx.id == fx.id:
          if self.verbose:
            print('[WARM]')
          self.warm_starts+=1
          return app,self.warm_start_delay
      if self.steal_warm_allowed == True:
        if self.verbose:
          print('[Steal WARM]')

        # Yes, it is cold. not warm
        self.cold_starts+=1
        return appropriate_warm_containers[0], self.steal_warm_delay
      else:
        return None,None
    else:
      return None,None

  def make_container(self, fx):
    if self.can_run(fx):
      c = Container(id=fx.id,ram=fx.ram,cpu=fx.cpu,gpu=fx.gpu,tpu=fx.tpu,runtime=fx.runtime,status='COLD')
      self.containers_queue.append(c)
      if self.verbose:
        print('[COLD]')

      self.cold_starts+=1
      return c,self.cold_start_delay
    else:
      return None,None

  def remove_container(self,c):
    self.functions_queue = [x for x in self.containers_queue if x.id != c.id]

  def convert(self,fx:Function,inplace = False):
    if self.conversion_allowed == False:
      return fx

    execution_time = fx.execution_time
    cpu = fx.cpu
    if fx.gpu>0:
      execution_time = execution_time * self.cpu_over_gpu
      cpu+= fx.gpu
    if fx.tpu>0:
      execution_time = execution_time * self.cpu_over_tpu
      cpu+=fx.tpu
    if inplace:
      fx.execution_time = execution_time
      fx.cpu = cpu
      fx.gpu = 0
      fx.tpu = 0
    else:
      gx = Function(t=fx.t,id=fx.id,execution_time=execution_time,ram=fx.ram,cpu=cpu,gpu=0,tpu=0,runtime=fx.runtime,priority=fx.priority,deadline=fx.deadline)
      return gx

  def can_run(self,fx):
    available_ram, available_cpu, available_gpu, available_tpu, = self.get_available_resources()
    return fx.ram <= available_ram and fx.cpu <= available_cpu and fx.gpu <= available_gpu and fx.tpu <= available_tpu


  def can_accept(self, fx):
    if fx.ram <= self.ram and fx.cpu <= self.cpu and fx.gpu <= self.gpu and fx.tpu <= self.tpu:
      return True
    else:
      return False

  def run(self,clock,c):
    if c.status=='COLD':

      self.used_ram+=c.ram
      self.used_cpu+=c.cpu
      self.used_gpu+=c.gpu
      self.used_tpu+=c.tpu

    if c.status=='WARM' and c.waited_clock!=None:
      c.execution_time += clock - c.waited_clock  
      
    c.status = 'HOT'


    return c.fx.execution_time

  def is_container_in_use(self,c):
    return c.status == 'HOT'


  def container_done(self,clock,event_list, c):
    fx = c.fx
    if c.status == 'HOT':
      # take care of the completed fx. just once!
      c.execution_time += fx.execution_time
      fx.finished_at = clock
      self.completed_functions.append(fx)
    
    if self.container_warm_time == 0 or c.status =='WARM':
      # timeout or no warm_time at all
      if c.waited_clock != None:
        c.execution_time += clock - c.waited_clock
      self.update_metrics(c,clock)
      self.used_ram-=c.ram
      self.used_cpu-=c.cpu
      self.used_gpu-=c.gpu
      self.used_tpu-=c.tpu
      self.completed_containers.append(c)
      self.containers_queue = [x for x in self.containers_queue if x.id!=c.id]
    else:
      c.status= 'WARM'
      # c.fx = None
      c.waited_clock = clock
      event_list [clock+self.container_warm_time] = ('LOCAL_CONTAINER_WARM_TIMEOUT',self,c)


  def show_status(self,clock):
    status = self.get_status(clock)
    print('='*(len(str(self))+4))
    print('|',self,'|')
    print('='*(len(str(self))+4))
    for k,v in status.items():
      print(' ',k,v)


  def get_status(self,clock):
    status=dict()

    running_containers = [c for c in self.containers_queue if c.status=='HOT']
    running_functions =  [c.fx for c in running_containers]
    missed_functions = self.missed_functions
    rejected_functions=self.rejected_functions
    # completed_functions = in-time completed + missed
    completed_functions= self.completed_functions
    completed_containers= self.completed_containers
    
    if clock>0:
      ramutil = self.ram_utilization/clock
      cpuutil = self.cpu_utilization/clock
      gpuutil = self.gpu_utilization/clock
      tpuutil = self.tpu_utilization/clock
      util =  {'RAM':round(ramutil,2),'CPU':round(cpuutil,2),'GPU':round(gpuutil,2),'TPU':round(tpuutil,2)}
      
    price = 0
    total_response_time = 0
    total_waiting_time  = 0
    total_service_time  = 0
    for f in completed_functions:
      price+= self.pricing_model(f)
      # R = W + S
      # W = R - S
      R = f.finished_at - f.t
      S = f.execution_time
      W = R - S
      total_response_time += R
      total_waiting_time +=  W
      total_service_time +=  S
    

    if len(completed_functions)>0:
      average_respone_time = total_response_time/ len(completed_functions)
      average_waiting_time = total_waiting_time / len(completed_functions)
      average_service_time = total_service_time / len(completed_functions)

    else:
      average_respone_time = 0
      average_waiting_time = 0
      average_service_time = 0

    status['average response time']=round(average_respone_time,2)
    status['average waiting time']=round(average_waiting_time,2)
    status['average service time']=round(average_service_time,2)

    status ['throughput'] = round(len(completed_functions)/clock,2)
    status ['in-time throughput'] = round((len(completed_functions)-len(missed_functions))/clock,2)

    status ['completed containers']=completed_containers
    status ['completed functions']=completed_functions
    status ['rejected functions']=rejected_functions
    status ['missed functions']=missed_functions 

    if (len(completed_functions)+len(rejected_functions))>0:
      status['completion ratio']=round(len(completed_functions)  /  (len(completed_functions)+len(rejected_functions))                                  ,2)
      status['in-time completion ratio']=round((len(completed_functions)-len(missed_functions))  /  (len(completed_functions)+len(rejected_functions))  ,2)
      status['rejection ratio']=round(len(rejected_functions)  /  (len(completed_functions)+len(rejected_functions))                                    ,2)
      status['missed ratio']=round(len(missed_functions)  /  (len(completed_functions)+len(rejected_functions))                                         ,2)
    else:
      status['completion ratio']=0
      status['intime completion ratio']=0
      status['rejection ratio']=0
      status['missed ratio']=0

    status['warm starts'] = self.warm_starts
    status['cold starts'] = self.cold_starts
    if self.warm_starts + self.cold_starts >0:
      status['warm start ratio'] = round(self.warm_starts / (self.warm_starts + self.cold_starts),2)
      status['cold start ratio'] = round(self.cold_starts / (self.warm_starts + self.cold_starts),2)
    else:
      status['warm start ratio'] = 0
      status['cold start ratio'] = 0

    status['utilization'] = util

    status['price'] = round(price,2)

    cost = 0
    for c in completed_containers:
      cost+= self.pricing_model(c) * self.cost_coeff
    status['cost'] = round(cost,2)


    penalty = 0
    for f in missed_functions:
        penalty += self.pricing_model(f)* self.penalty_coeff
    for f in rejected_functions:
        penalty += self.pricing_model(f)* self.penalty_coeff
    status['penalty']=round(penalty,2)

    status['loss']=round(cost+penalty-price,2)


    return status


  def update_metrics(self,c,clock):
    if clock > 0:
      
      self.ram_utilization = self.utilization(self.ram_utilization,c.ram,self.ram,c.execution_time)
      self.cpu_utilization = self.utilization(self.cpu_utilization,c.cpu,self.cpu,c.execution_time)
      self.gpu_utilization = self.utilization(self.gpu_utilization,c.gpu,self.gpu,c.execution_time)
      self.tpu_utilization = self.utilization(self.tpu_utilization,c.tpu,self.tpu,c.execution_time)

  def get_current_utilization(self):
    num_resource_type = 0
    total_util = 0
    if self.ram> 0:
      num_resource_type+=1
      ram_util = self.used_ram / self.ram
      total_util+=ram_util

    if self.cpu> 0:
      num_resource_type+=1
      cpu_util = self.used_cpu / self.cpu
      total_util+=cpu_util

    if self.gpu> 0:
      num_resource_type+=1
      gpu_util = self.used_gpu / self.gpu
      total_util+=gpu_util
    
    if self.tpu> 0:
      num_resource_type+=1
      tpu_util = self.used_tpu / self.tpu
      total_util+=tpu_util

    return total_util/num_resource_type

  # Models
  def utilization(self,old_utilization_value,used_amount,total_amount,duration):
    try:
      now_util = used_amount / total_amount
      timed_now_util = now_util * duration
      new_util = old_utilization_value + timed_now_util
      return new_util
    except:
      return 0


  def pricing_model(self,unit):
    ram = unit.ram * unit.execution_time * self.ram_unit_price
    cpu = unit.cpu * unit.execution_time * self.cpu_unit_price
    gpu = unit.gpu * unit.execution_time * self.gpu_unit_price
    tpu = unit.tpu * unit.execution_time * self.tpu_unit_price
    total = ram + cpu + gpu + tpu
    return total



  def local_scheduler(self,clock,event_list):
    if self.verbose:
      print(f'[Scheduling in PM {self.id}]')
    fx_list = self.functions_queue
    for fx in fx_list:
      # Warm start
      c,delay = self.get_warm_container(fx)

      
      if c == None:
      # Cold start
        c,delay = self.make_container(fx)

      if c== None:
        # Try to change the hardware
        gx = self.convert(fx)
        # Warm start
        c,delay = self.get_warm_container(gx)    
        if c == None:
        # Cold start
          c,delay = self.make_container(gx)

        if c!= None:
          if self.verbose:
            print('[Converting...]')
          self.convert(fx,True)
      
      if c == None:
      # Not enough resources
        if self.verbose:
          print('Not enough resources')
        
        if self.can_accept(fx) or self.can_accept(self.convert(fx)):
        # Run later
          first_event_function_done = event_list.first_event('LOCAL_FUNCTION_DONE',self)
          first_event_container_warm_timeout = event_list.first_event('LOCAL_CONTAINER_WARM_TIMEOUT',self)
          if first_event_function_done == None:
            first_event = first_event_container_warm_timeout
          elif first_event_container_warm_timeout == None:
            first_event = first_event_function_done
          else:
            first_event = min(first_event_function_done, first_event_container_warm_timeout)
          
          if self.verbose:
            print(fx,'postponed until',first_event)

          first_event_FUNCTION_SCHEDULE = event_list.first_event('LOCAL_FUNCTION_SCHEDULE',self)
          if first_event_FUNCTION_SCHEDULE == None:
            event_list [first_event] = ('LOCAL_FUNCTION_SCHEDULE',self) 

        else:
          # Reject
          self.remove_function(fx)
          self.rejected_functions.append(fx)
          if self.verbose:
            print("can't accept")

      else:
      # Run now
        c.inject(fx)
        if self.verbose:
          print(fx,'->',c)

        t = self.run(clock,c)
        self.remove_function(fx)
        run_clock = clock+delay+t
        if run_clock> fx.deadline:
          self.missed_functions.append(fx)
        event_list[run_clock] = ('LOCAL_FUNCTION_DONE',self,c)

## Simulator (& Controller)

In [ ]:
from copy import deepcopy
class Simulator:
  def __init__(self,physical_machines:list,functions):
    # Parameters
    self.tmp_p = physical_machines
    self.tmp_f = functions
    self.f_counter = 0

  def function_arrival(self,next_event,clock,event_list):
    # Global Scheduler
    p = self.p
    verbose = self.verbose
    global_scheduling_strategy = self.global_scheduling_strategy

    fx = next_event[1]
    if global_scheduling_strategy == 'loadbalancing':
      pm_index = self.f_counter % len(p)
      self.f_counter +=1
      pm = p[pm_index]
    elif global_scheduling_strategy == 'utilization':
      pm = p [0]
      utilized_pms = [pm for pm in p]
      utilized_pms = sorted(utilized_pms,key= lambda x:x.get_current_utilization(),reverse=True)

      for pmx in utilized_pms:
        if pmx.can_accept(fx) or pmx.can_accept(pmx.convert(fx)):
          pm = pmx
          break
    else:
      raise Exception('Invalid strategy!')

    if verbose:
      print(pm)
    
    pm.add_function(fx)
    pm.local_scheduler(clock,event_list)

  def local_function_schedule(self,next_event,clock,event_list):
    p = self.p
    verbose = self.verbose

    pm = next_event[1]
    if verbose:
      print(pm)

    pm.local_scheduler(clock,event_list)
    
  def local_function_done(self,next_event,clock,event_list):
    p = self.p
    verbose = self.verbose

    pm = next_event[1]
    if verbose:
      print(pm)

    container = next_event[2]
    pm.container_done(clock,event_list,container)

  def local_container_warm_timeout(self,next_event,clock,event_list):
    p = self.p
    verbose = self.verbose

    pm = next_event[1]
    if verbose:
      print(pm)
    container = next_event[2]

    if not pm.is_container_in_use(container):
      pm.container_done(clock,event_list,container)
      if verbose:
        print('container removed')
    else:
      if verbose:
        print('The container is in use. ignoring TIMEOUT...')


  def show_statistics(self,verbose = True):
    parameters = self.parameters
    print('Experiment parameters:')
    for k,v in parameters.items():
      print('{:<30}  {:>30}'.format(k, v))

    print('_'*62)
    result = self.get_statistics()
    for k,v in result.items():
      if type(k)==PM:
        if verbose:
          print('='*(len(str(k))+4))
          print('|',k,'|')
          print('='*(len(str(k))+4))

          for kp,vp in v.items():
            print(' ',kp,vp)
          print('_'*62)
      elif type(v)==dict:
        print(k)
        for kp,vp in v.items():
          print('  {:<28}  {:>30}'.format(kp, vp))
      else:
        print('{:<30}  {:>30}'.format(k, v))

  def get_statistics(self):
    result =dict()
    total_functions = len(self.f)

    total_cold_starts = 0
    total_warm_starts = 0

    total_response_time = 0
    total_waiting_time  = 0
    total_service_time  = 0

    len_completed = 0
    len_missed =0
    len_rejected=0

    price = 0
    cost = 0
    penalty =0
    ram_util = 0
    cpu_util = 0
    gpu_util = 0
    tpu_util = 0
    for pm in self.p:
      status = pm.get_status(self.clock)
      result[pm]=status

      cold_starts = status ['cold starts']
      warm_starts = status ['warm starts']

      total_cold_starts += cold_starts
      total_warm_starts += warm_starts

      completed_containers=status['completed containers']
      completed_functions = status['completed functions']
      len_completed+=len(completed_functions)
      len_missed+=len(status['missed functions'])
      len_rejected+=len(status['rejected functions'])
      

      total_response_time+=status['average response time']*len(completed_functions)
      total_waiting_time +=status['average waiting time']*len(completed_functions)
      total_service_time +=status['average service time']*len(completed_functions)

      price += status['price']
      cost += status['cost']
      penalty+=status['penalty']

      if 'utilization' in status:
        util = status['utilization']
        ram_util += util['RAM']
        cpu_util += util['CPU']
        gpu_util += util['GPU']
        tpu_util += util['TPU']

    result['total time'] = self.clock
    if len_completed>0:
      result['average response time'] = round(total_response_time/len_completed,2)
      result['average waiting time'] = round(total_waiting_time/len_completed,2)
      result['average service time'] = round(total_service_time/len_completed,2)

    else:
      result['average response time'] = 0
      result['average response time'] = 0
      result['average response time'] = 0

    result['throughput'] = round(len_completed / self.clock,2)
    result['in-time throughput']=round((len_completed-len_missed) / self.clock,2)

    if total_functions>0:
      result['completion ratio']=round(len_completed  /  total_functions                        ,2)
      result['in-time completion ratio']=round((len_completed-len_missed)  /  total_functions   ,2)
      result['rejection ratio']=round(len_rejected  /  total_functions                          ,2)
      result['missed ratio']=round(len_missed  /  total_functions                               ,2)
    else:
      result['completion ratio']=0
      result['intime completion ratio']=0
      result['rejection ratio']=0
      result['missed ratio']=0

    if total_cold_starts + total_warm_starts >0:
      result['warm start ratio'] = round(total_warm_starts / (total_cold_starts + total_warm_starts),2)
      result['cold start ratio'] = round(total_cold_starts / (total_cold_starts + total_warm_starts),2)
    else:
      result['warm start ratio'] = 0
      result['cold start ratio'] = 0

    util = dict()
    util['RAM']=round(ram_util/len(self.p),2)
    util['CPU']=round(cpu_util/len(self.p),2)
    util['GPU']=round(gpu_util/len(self.p),2)
    util['TPU']=round(tpu_util/len(self.p),2)
    result['utilization'] = util

    result['price']=round(price,2)
    result['cost']=round(cost,2)
    result['penalty']=round(penalty,2)
    result['loss']=round(cost+penalty-price,2)

    return result

  def run(self,container_warm_time = 2, cold_start_delay=2,warm_start_delay=0, steal_warm_delay=1,
              steal_warm_allowed = False, conversion_allowed=False, 
              cpu_over_gpu=2,cpu_over_tpu=4, ram_unit_price = 21, cpu_unit_price = 0, gpu_unit_price = 0, tpu_unit_price = 0,
              global_scheduling_strategy ='utilization',cost_coeff = 0.8,penalty_coeff = 4 , verbose=False):
    
    self.p = deepcopy(self.tmp_p)
    self.f = deepcopy(self.tmp_f)
    
    parameters = dict()
    parameters['container warm time'] = container_warm_time
    parameters['cold start delay'] = cold_start_delay
    parameters['warm start delay'] = warm_start_delay
    parameters['steal warm delay'] = steal_warm_delay

    parameters['conversion allowed'] = conversion_allowed
    parameters['steal warm allowed'] = steal_warm_allowed

    parameters['cpu over gpu'] = cpu_over_gpu
    parameters['cpu over tpu'] = cpu_over_tpu
    parameters['ram unit price'] = ram_unit_price
    parameters['cpu unit price'] = cpu_unit_price
    parameters['gpu unit price'] = gpu_unit_price
    parameters['tpu unit price'] = tpu_unit_price

    parameters['global scheduling strategy'] = global_scheduling_strategy
    parameters['cost coeff'] = cost_coeff
    parameters['penalty coeff'] = penalty_coeff
    parameters['verbose'] = verbose
    
    self.parameters = parameters

    self.event_list = Event_List()
    
    for item in self.f:
      i = item.t
      self.event_list[i]=('FUNCTION_ARRIVAL',item)

    self.clock = 0
    self.container_warm_time = container_warm_time
    self.cold_start_delay = cold_start_delay
    self.warm_start_delay = warm_start_delay
    self.steal_warm_delay = steal_warm_delay

    self.cpu_over_gpu = cpu_over_gpu
    self.cpu_over_tpu = cpu_over_tpu
    self.ram_unit_price=ram_unit_price
    self.cpu_unit_price=cpu_unit_price
    self.gpu_unit_price=gpu_unit_price

    self.tpu_unit_price = tpu_unit_price
    self.global_scheduling_strategy = global_scheduling_strategy
    self.verbose = verbose

    for pm in self.p:
      pm.t = 0
      pm.container_warm_time = container_warm_time
      pm.cold_start_delay = cold_start_delay
      pm.warm_start_delay = warm_start_delay
      pm.steal_warm_delay = steal_warm_delay
      pm.cpu_over_gpu = cpu_over_gpu
      pm.cpu_over_tpu = cpu_over_tpu    

      pm.ram_unit_price = ram_unit_price
      pm.cpu_unit_price = cpu_unit_price
      pm.gpu_unit_price = gpu_unit_price
      pm.tpu_unit_price = tpu_unit_price
      pm.cost_coeff = cost_coeff
      pm.penalty_coeff = penalty_coeff
      pm.conversion_allowed = conversion_allowed
      pm.steal_warm_allowed = steal_warm_allowed
      pm.verbose = verbose

    event_list = self.event_list


    
    while not event_list.is_empty():
      if self.verbose:
        print('------------------------------')
      clock,next_event = event_list.pop()
      self.clock = clock
      
      if self.verbose:
        print('Clock =',clock)
        print('Event =',next_event)
      
      e_type = next_event[0]
      if e_type =='FUNCTION_ARRIVAL':
        self.function_arrival(next_event,clock,event_list)

      elif e_type =='LOCAL_FUNCTION_SCHEDULE':
        self.local_function_schedule(next_event,clock,event_list)

      elif e_type=='LOCAL_FUNCTION_DONE':
        self.local_function_done(next_event,clock,event_list)

      elif e_type=='LOCAL_CONTAINER_WARM_TIMEOUT':
        self.local_container_warm_timeout(next_event,clock,event_list)

# Playground

Example One - Simple

In [38]:

#111111111
p = [
     PM(id=0,ram=1,cpu=1)
]
experiment=7
f1 = [
      Function(id=0,t=i,execution_time=5,ram=1,cpu=1,gpu=0,tpu=0,deadline=300) for i in range(0,experiment)
  ]
f2 = [
      Function(id=i,t=i,execution_time=5,ram=1,cpu=1,gpu=0,tpu=0,deadline=300) for i in range(experiment,10)
  ]
f=f1+f2

s = Simulator(physical_machines=p,functions=f)
s.run(conversion_allowed=False,verbose=False,global_scheduling_strategy='loadbalancing',steal_warm_allowed=False,warm_start_delay = 1,steal_warm_delay=2,cold_start_delay=3)
s.show_statistics(verbose=True)
#s.p[0].show_status(s.get_statistics()['total time'])

Experiment parameters:
container warm time                                          2
cold start delay                                             3
warm start delay                                             1
steal warm delay                                             2
conversion allowed                                           0
steal warm allowed                                           0
cpu over gpu                                                 2
cpu over tpu                                                 4
ram unit price                                              21
cpu unit price                                               0
gpu unit price                                               0
tpu unit price                                               0
global scheduling strategy                       loadbalancing
cost coeff                                                 0.8
penalty coeff                                                4
verbose                         

Example two

In [ ]:
# Containers
container_warm_time = 2
cold_start_delay = 4
warm_start_delay = 0
steal_warm_delay = 0

# Time
cpu_over_gpu = 2
cpu_over_tpu = 4

# Price
ram_unit_price = 1
cpu_unit_price = 1
gpu_unit_price = 1.5
tpu_unit_price = 2

# Cost
cost_coeff = 0.8

# Penalty
penalty_coeff = 2

# Misc
avilable_strategies = ['utilization','loadbalancing']
global_scheduling_strategy = avilable_strategies[0]
conversion_allowed = True
steal_warm_allowed = True
verbose = True

p = [
     PM(id=0,ram=4,cpu=4,gpu=0,tpu=0),
     #PM(id=1,ram=4,cpu=4,gpu=4,tpu=0),
]

f = [
     Function(id=0,t=0,execution_time=2,ram=1,cpu=0,gpu=0,tpu=0,runtime='python',deadline=300),
     Function(id=1,t=0,execution_time=2,ram=1,cpu=0,gpu=0,tpu=0,runtime='python',deadline=300),
     Function(id=2,t=1,execution_time=5,ram=2,cpu=2,gpu=0,tpu=0,runtime='python',deadline=300),
     Function(id=3,t=1,execution_time=3,ram=2,cpu=2,gpu=0,tpu=0,runtime='python',deadline=300),
     Function(id=4,t=2,execution_time=1,ram=2,cpu=4,gpu=0,tpu=0,runtime='python',deadline=300),
     Function(id=5,t=5,execution_time=8,ram=4,cpu=3,gpu=0,tpu=0,runtime='python',deadline=300),

     #Function(id=1,t=9,execution_time=2,ram=1,cpu=1,gpu=0,tpu=0,runtime='python',deadline=300),
]

s = Simulator(physical_machines=p,functions=f)
s.run(container_warm_time = container_warm_time, cold_start_delay=cold_start_delay,warm_start_delay=warm_start_delay, steal_warm_delay=steal_warm_delay,
              steal_warm_allowed = steal_warm_allowed, conversion_allowed=conversion_allowed, 
              cpu_over_gpu=cpu_over_gpu,cpu_over_tpu=cpu_over_tpu, ram_unit_price = ram_unit_price, cpu_unit_price = cpu_unit_price, gpu_unit_price = gpu_unit_price, tpu_unit_price = tpu_unit_price,
              global_scheduling_strategy =global_scheduling_strategy,cost_coeff = cost_coeff,penalty_coeff = penalty_coeff , verbose=verbose)
s.show_statistics(verbose)

------------------------------
Clock = 0
Event = ('FUNCTION_ARRIVAL', Function(id=0,RAM=1,CPU=0,GPU=0,TPU=0,Entered at 0,Finished at None,Execution time=2,Deadline=300,Runtime=python))
PM(id=0,RAM=4,CPU=4,GPU=0,TPU=0)
[Scheduling in PM 0]
[COLD]
Function(id=0,RAM=1,CPU=0,GPU=0,TPU=0,Entered at 0,Finished at None,Execution time=2,Deadline=300,Runtime=python) -> Container(id=0,RAM=1,CPU=0,GPU=0,TPU=0,Execution time=0,Function(id=0,RAM=1,CPU=0,GPU=0,TPU=0,Entered at 0,Finished at None,Execution time=2,Deadline=300,Runtime=python))
------------------------------
Clock = 0
Event = ('FUNCTION_ARRIVAL', Function(id=1,RAM=1,CPU=0,GPU=0,TPU=0,Entered at 0,Finished at None,Execution time=2,Deadline=300,Runtime=python))
PM(id=0,RAM=4,CPU=4,GPU=0,TPU=0)
[Scheduling in PM 0]
[COLD]
Function(id=1,RAM=1,CPU=0,GPU=0,TPU=0,Entered at 0,Finished at None,Execution time=2,Deadline=300,Runtime=python) -> Container(id=1,RAM=1,CPU=0,GPU=0,TPU=0,Execution time=0,Function(id=1,RAM=1,CPU=0,GPU=0,TPU=0,Entered a